In [2]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
import pyjsonviewer
load_dotenv()

True

In [3]:
client_id = os.getenv("tok1")
client_secret = os.getenv("tok2")
dondebuscamos = geocode("Paseo de la chopera 14, Madrid")
url_query = 'https://api.foursquare.com/v2/venues/explore'
parametros = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{dondebuscamos['coordinates'][0]}, {dondebuscamos['coordinates'][1]}",
    "query": "Frutería",
    "limit": 200   
}
resp = requests.get(url_query, params=parametros).json()

In [ ]:
pyjsonviewer.view_data(json_data=resp)

In [4]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost/companies")
db=client.get_database("companies")
c = db.get_collection("companies")


In [ ]:
from pymongo import GEOSPHERE

In [ ]:
query = [{
        '$geoNear': {'near': [la_lat_lon[1],la_lat_lon[0]],
            'distanceField': 'distancia',
            'maxDistance': 10000,
            'query': { 'nombre': {'$regex':'Starbucks' }},
            'distanceMultiplier': 6371.137,
            #'includeLocs': "dist.location",
            'spherical': True,
            }
        }]

In [ ]:
db.barrios.create_index([("geometry", GEOSPHERE)])
b = db.get_collection("barrios")

In [8]:
proj ={"name":1, "offices.country_code":1,"offices.latitude":1,
       "offices.longitude":1,"_id":0}
cond1 ={"tag_list":{"$regex": "games"}}
cond2 ={"total_money_raised": {"$regex":"\d.M"}}
seeds = list(c.find({"$and":[cond1,cond2]}, proj).sort("total_money_raised",-1))

In [12]:
len(seeds)

16

In [14]:
for n,seed in enumerate(seeds):
    for i,office in enumerate(seed["offices"]):
        if office["latitude"] == None:
            del seed["offices"][i]
        if len(seed["offices"])==0:
            del seeds[n]
seeds

[{'name': 'Bigpoint',
  'offices': [{'country_code': 'DEU',
    'latitude': 53.5973055,
    'longitude': 9.9762519}]},
 {'name': 'eRepublik',
  'offices': [{'country_code': 'ESP',
    'latitude': 40.43094,
    'longitude': -3.6952896}]},
 {'name': 'Apaja',
  'offices': [{'country_code': 'FIN',
    'latitude': 60.171821,
    'longitude': 24.945167}]},
 {'name': 'Zynga',
  'offices': [{'country_code': 'USA',
    'latitude': 37.765158,
    'longitude': -122.404234}]},
 {'name': 'WildTangent',
  'offices': [{'country_code': 'USA',
    'latitude': 37.09024,
    'longitude': -95.712891}]},
 {'name': 'Heyzap',
  'offices': [{'country_code': 'USA',
    'latitude': 37.7905538,
    'longitude': -122.4041485}]},
 {'name': 'Metacafe',
  'offices': [{'country_code': 'USA',
    'latitude': 37.437328,
    'longitude': -122.159928}]},
 {'name': 'Zango',
  'offices': [{'country_code': 'USA',
    'latitude': 47.579208,
    'longitude': -122.155668}]},
 {'name': 'Hands-On Mobile',
  'offices': [{'country

In [1]:
def geocode(direccion):
    """
    Esta función saca las coordenadas de la dirección que le pases
    """
    data = requests.get(f"https://geocode.xyz/{direccion}?json=1").json()
    try:
        return {
            "type": "Point",
            "coordinates": [data["latt"], data["longt"]]
        }
    
    except:
        return data